In [ ]:
import sys, os, torch, warnings
import copy
sys.path.append(os.path.abspath('..'))
from trainers.train import get_glue_res, make_dataset, get_distill_res
from trainers.seed import seedval, seeds
from trainers.args import *
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device = {device}')
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

dataset = make_dataset(tasks)
res = {}
l_values = {'rte': {0:1.0,42:1.3,777:1.0},'mrpc': {0:1.1,42:1.3,777:1.1},'sst2': {0:1.1,42:1.2,777:1.0}}
distill_values = [1,2,3]

for seed in seeds : 
    seedval(seed)
    res[seed] = {}
    for task in tasks :
        arg = copy.deepcopy(args)
        kwarg1 = copy.deepcopy(baseline)
        kwarg2 = copy.deepcopy(ours)
        arg['seed'] = seed
        
        if task == 'boolq' : 
            kwarg1['sl'], kwarg2['sl'] = 256, 256
            arg['batch_size'],arg['per_device_eval_batch_size'] = 32,64
        
        res[seed][task] = []
        model, evals = get_glue_res(dataset[task], task, arg, **kwarg1)
        kwarg2['l'] = l_values[task][seed]
        
        for distill in distill_values :
            kwarg2['distill'] = distill
            loss,evals2 = get_distill_res(dataset[task], task, arg, model, **kwarg2)
            res[seed][task].append((evals,loss,evals2))
            print(res[seed][task][-1])
                

print('-'*150)
print('results')
explain = {1:'softmax_only',2:'ln_only',3:'both'}

for seed in seeds :
    for task in tasks :
        for i,distill in enumerate(distill_values) :
            print(f'seed : {seed}, task : {task}, distill : {explain[distill]}, res : orig {res[seed][task][i][0]*100:.2f} | loss {res[seed][task][i][1]:.2f} | distill {res[seed][task][i][2]*100:.2f}')